## top_template

## ソースコード

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/article/library/scipy/template/template_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/article/library/scipy/template/template_nb.ipynb)


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/graph/01/01_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/graph/01/01_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なることに注意。

In [ ]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [ ]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認する。
ついでに乱数のseedの設定を行う。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
import random

import scipy
import numpy as np


import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



# 第5回：コレクション（リスト、タプル、セット、辞書）の使い方

## **目次**
1. はじめに
2. リスト（list）
   - 2.1 リストの基本
   - 2.2 リストの操作（追加・削除・ソート等）
   - 2.3 リストを使った大量データのバッチ処理（企業利用例）
   - 2.4 リストのメリット・デメリット
3. タプル（tuple）
   - 3.1 タプルの基本
   - 3.2 タプルの利用例（不変データの管理）
   - 3.3 タプルのメリット・デメリット
4. セット（set）
   - 4.1 セットの基本（重複削除の仕組み）
   - 4.2 集合演算（union, intersection, difference）
   - 4.3 セットを用いた重複排除や要素管理（企業利用例）
   - 4.4 セットのメリット・デメリット
5. 辞書（dict）
   - 5.1 辞書の基本（キーと値）
   - 5.2 辞書の操作（追加・削除・更新）
   - 5.3 代表的なメソッド（keys, values, items, get, pop等）
   - 5.4 企業利用例（DBレコード管理、JSONとの親和性）
   - 5.5 辞書のメリット・デメリット
6. Python 3.11系でのコレクション周りの注意点・改良点
   - 6.1 エラーメッセージの改良
   - 6.2 内部実装のパフォーマンス
7. コレクション混在の応用例
   - 7.1 リストの中に辞書を入れる
   - 7.2 辞書の値としてリストやタプルを保持する
   - 7.3 現場でよくあるデータ構造パターン
8. 具体的なコード例（大量）
   - 8.1 リスト例
   - 8.2 タプル例
   - 8.3 セット例
   - 8.4 辞書例
   - 8.5 応用例：コレクションを使った簡単なアプリケーション
9. 演習問題
   - 9.1 問題文
   - 9.2 解答例
10. まとめ
    - 10.1 コレクションを使いこなすポイント
    - 10.2 今後の学習・応用について
11. 参考文献・参考URL

---

## **1. はじめに**

### **前回の振り返りと本講座の位置づけ**
前回（第4回）では、**制御構文**（`if`, `elif`, `else` など条件分岐や、`for`, `while` などループ処理、`range()`、`break`, `continue`など）を学びました。制御構文によってプログラムの流れを自在に操り、複雑な処理を記述できるようになりました。

しかし実際のプログラミングにおいては、制御構文だけでなく、さまざまな**データ構造**を使いこなす必要があります。たとえば、「大量の顧客情報を扱う」「ユーザーから入力されたデータを管理する」「レコメンドエンジンでユーザーの購入履歴をまとめる」など、**一時的にも永続的にも、多様なデータを格納・操作する**場面が非常に多いのです。

Pythonでは、**リスト（list）・タプル（tuple）・セット（set）・辞書（dict）**という4種類のコレクションを中心に、さらに高度なデータ構造やライブラリが存在します。本講座では、まずこれら4種類のコレクションの特徴・使い分け・操作方法を理解することで、**実務でも頻出する問題解決**の基礎を習得します。

### **Python 3.11系の特徴的ポイント**
- Python 3.7以降で辞書（dict）のキー順序保持が標準仕様になり、3.11系でも順序保持が当然のように使えます。
- 3.11系ではエラーメッセージがさらに改善されており、コレクションの操作時にありがちな型エラーやインデックスエラーが出た場合にも、原因をより特定しやすくなっています。
- 内部実装のパフォーマンスも少しずつ向上しており、とくに辞書やセットのハッシュテーブル周りの処理が高速化しているケースがあります。

こうしたポイントも踏まえつつ、各コレクションの使い方を具体的なコード例とともに解説します。

---

## **2. リスト（list）**

### **2.1 リストの基本**
リスト（list）は、Pythonにおける**可変長（mutable）**のシーケンス型です。リストの要素には、整数や文字列、他のリスト、辞書など、Pythonで扱えるすべてのオブジェクトを格納できます。要素の順序があり、同じ要素を重複して入れることも可能です。

#### **リストの宣言方法**
1. **リテラル表記**（中括弧 `[ ]` を使う）
   ```python
   fruits = ["apple", "banana", "orange"]
   numbers = [10, 20, 30, 40, 50]
   mixed = [1, "two", 3.0, True, [10, 20], {"key": "value"}]
   ```
2. **list() コンストラクタ**
   ```python
   data = list(range(5))  # [0, 1, 2, 3, 4]
   ```
3. **空のリスト**
   ```python
   empty_list = []
   another_empty = list()
   ```

#### **要素へのアクセス**
- インデックスを使ってアクセス（0-based index）
  ```python
  fruits = ["apple", "banana", "orange"]
  print(fruits[0])  # apple
  print(fruits[2])  # orange
  ```
- スライス（slice）で部分リストを取得
  ```python
  print(fruits[0:2])  # ["apple", "banana"]
  print(fruits[:2])   # 上と同じ
  print(fruits[1:])   # ["banana", "orange"]
  ```

### **2.2 リストの操作**

#### **要素の追加**
- `append(value)`: リスト末尾に1つの要素を追加
  ```python
  fruits.append("grape")
  print(fruits)  # ["apple", "banana", "orange", "grape"]
  ```
- `extend(iterable)`: リスト末尾に別のイテラブル（リストやタプルなど）の要素を展開して追加
  ```python
  more_fruits = ["peach", "melon"]
  fruits.extend(more_fruits)
  print(fruits)
  # ["apple", "banana", "orange", "grape", "peach", "melon"]
  ```
- スライスでの代入による複数要素の挿入も可能
  ```python
  # fruits[2:2] = ["pineapple", "kiwi"] など
  ```

#### **要素の挿入**
- `insert(index, value)`: 指定したインデックス位置に要素を挿入
  ```python
  fruits.insert(1, "lemon")
  print(fruits)  # ["apple", "lemon", "banana", "orange", ...]
  ```

#### **要素の削除**
- `remove(value)`: リスト内で最初に見つかった`value`を削除
  ```python
  fruits.remove("banana")
  # "banana" が先に見つかれば削除
  ```
- `pop(index)`: 指定インデックスの要素を取り出して返す（指定しなければ末尾をpop）
  ```python
  last_item = fruits.pop()
  print(last_item)   # 末尾の要素
  first_item = fruits.pop(0)  # 先頭の要素
  ```
- `del fruits[index]`: 組み込み命令 `del` による削除
  ```python
  del fruits[1]  # 1番目の要素を削除
  ```

#### **要素の検索**
- `in` 演算子を用いると、リスト内に要素が含まれているかを判定
  ```python
  if "apple" in fruits:
      print("リンゴがあります")
  ```
- `list.index(value)`: 該当要素のインデックスを返す（存在しない場合ValueError）
  ```python
  idx = fruits.index("orange")
  print(idx)
  ```

#### **ソートとリバース**
- `list.sort()`: リストをインプレースでソート（元のリストが変更される）
  ```python
  numbers = [5, 2, 9, 1]
  numbers.sort()
  print(numbers)  # [1, 2, 5, 9]
  ```
- `sorted(iterable)`: 新しいリストとしてソート結果を返す（元のリストは変更されない）
  ```python
  numbers = [5, 2, 9, 1]
  sorted_numbers = sorted(numbers)
  print(numbers)         # [5, 2, 9, 1]
  print(sorted_numbers)  # [1, 2, 5, 9]
  ```
- `list.reverse()`: リストを逆順にする（インプレース）
- `reversed(iterable)`: 逆順イテレータを返す（新しいリストにしたい場合は `list(reversed(iterable))` とする）

### **2.3 リストを使った大量データのバッチ処理（企業利用例）**
企業では、大量のIDやレコードをリストに格納して、**バッチ処理**を行うケースが多々あります。

例：ユーザーIDのリストを用いて、一括でメールを送信する処理
```python
user_ids = [101, 102, 103, 104, ...]  # 数千、数万件ある場合も

for uid in user_ids:
    # それぞれのユーザーIDに対してメール送信処理
    send_email_to_user(uid)
```

データサイエンスの現場でも、CSVファイルやデータベースから取得したレコードをリストとして受け取り、その後ループで**前処理**や**加工**をするケースが典型的です。また、**機械学習**の前処理として、テキストをトークンに分割してリスト化したり、レコメンドの対象アイテムを一括で処理したりと、非常に幅広く利用されます。

### **2.4 リストのメリット・デメリット**
- **メリット**
  - 順序があり、追加・削除・ソートが簡単に行える
  - 同じ要素を重複して格納可能（listに含まれる要素の重複を許す）
  - ほぼすべてのシチュエーションで利用可能で、汎用性が高い
- **デメリット**
  - 可変長のため、誤って要素を変更・削除してしまうリスクがある
  - 大きなリストを操作するとき、特定の操作（リスト先頭への挿入等）に時間がかかる可能性
  - 大量データを扱う際にはメモリ消費が大きくなる場合がある

---

## **3. タプル（tuple）**

### **3.1 タプルの基本**
タプル（tuple）は、リストと同様に順序をもったシーケンス型ですが、**イミュータブル（immutable）**（変更不可）という点が最大の特徴です。要素を追加・削除・変更する操作はできません。

#### **タプルの作成方法**
1. **リテラル表記**（丸括弧 `()` を使う）
   ```python
   colors = ("red", "green", "blue")
   ```
2. **コンマ演算**: カッコを省略して書くことも可能
   ```python
   numbers = 1, 2, 3
   ```
3. **tuple() コンストラクタ**
   ```python
   another_tuple = tuple([10, 20, 30])
   ```

#### **要素へのアクセス**
- リストと同じくインデックスでアクセス可能
  ```python
  print(colors[0])  # red
  print(colors[1])  # green
  ```
- スライスも可能
  ```python
  print(colors[0:2])  # ("red", "green")
  ```

#### **変更不可**
```python
colors[1] = "yellow"
# TypeError: 'tuple' object does not support item assignment
```
このように、タプルの要素は変更しようとするとエラーが発生します。

### **3.2 タプルの利用例（不変データの管理）**
- **座標や日付など、変更を前提としない複数値**をひとまとめにする
  例：`coordinate = (10, 20)` のように2次元座標をタプルで表現
- **辞書のキー**としてタプルを使う
  - 辞書のキーにはイミュータブルなオブジェクトだけが利用できる（Python 3.11系でも同様）。タプルは変更不可なので、キーにできる。
  - 例：`dict_key = {(10, 20): "PointA", (30, 40): "PointB"}`
- **関数からの複数値返却**
  - Pythonでは`return`で1つのタプルを返すことが多い。実際にはタプルをアンパックして受け取るのも典型的。
  ```python
  def calculate(a, b):
      return (a + b, a * b)

  sum_val, mul_val = calculate(3, 5)
  print(sum_val, mul_val)  # 8, 15
  ```

### **3.3 タプルのメリット・デメリット**
- **メリット**
  - 不変性（immutable）により、安全かつ予期せぬ変更が起こりにくい
  - 辞書のキーなど変更不可オブジェクトが求められる場面で有効
  - リストよりも若干軽量で高速に操作できることが多い
- **デメリット**
  - 一度作成したタプルの要素は変更できないため、柔軟なデータ操作には不向き
  - 要素数が大きい場合、実用性が下がる（更新の必要があるならリストや別の構造を検討する）

---

## **4. セット（set）**

### **4.1 セットの基本（重複削除の仕組み）**
セット（set）は、**重複を許さない**、**順序のない**コレクションです。数学的な「集合」と同様に扱われ、要素が一意でなければならないため、重複を自動的に排除してくれます。

#### **セットの作成方法**
1. **波括弧 `{ }`** に要素をコンマ区切りで並べる
   ```python
   unique_values = {1, 2, 3, 2}
   print(unique_values)  # {1, 2, 3}
   ```
2. **set() コンストラクタ**
   ```python
   fruits = set(["apple", "banana", "apple"])
   print(fruits)  # {"apple", "banana"}
   ```
3. **空のセットを作る場合は、`{}` ではなく `set()`**
   ```python
   empty_set = set()
   # {} は空の辞書として扱われるので注意
   ```

#### **要素の追加・削除**
- `add(value)`: セットに要素を追加（すでに存在する場合は何もしない）
  ```python
  s = {1, 2, 3}
  s.add(3)
  s.add(4)
  print(s)  # {1, 2, 3, 4}
  ```
- `remove(value)`: 指定要素を削除（存在しない場合KeyError）
  ```python
  s.remove(2)
  print(s)  # {1, 3, 4}
  # s.remove(5)  # KeyError
  ```
- `discard(value)`: 要素を削除（存在しなくてもエラーにならない）
  ```python
  s.discard(10)  # エラーにならない
  ```
- `pop()`: ランダムに1つの要素を取り除いて返す（順序がないため取り除かれる要素が予測不能）
- `clear()`: 全要素を削除

### **4.2 集合演算（union, intersection, difference）**
セットは数学的な集合演算をサポートしており、**大量のデータから共通部分を取り出す**・**重複をまとめる**などの処理に役立ちます。

```python
a = {1, 2, 3, 4}
b = {3, 4, 5, 6}

# 和集合（union）
print(a | b)               # {1, 2, 3, 4, 5, 6}
print(a.union(b))          # 上と同じ

# 積集合（intersection）
print(a & b)               # {3, 4}
print(a.intersection(b))   # 上と同じ

# 差集合（difference）
print(a - b)               # {1, 2}
print(a.difference(b))     # 上と同じ

# 排他的論理和（symmetric_difference）
print(a ^ b)               # {1, 2, 5, 6}
print(a.symmetric_difference(b))
```

### **4.3 セットを用いた重複排除や要素管理（企業利用例）**
- **重複排除**：膨大なユーザーデータにおいて、同じユーザーIDが重複して入っている場合でも、セットに格納すれば一意な集合だけ残る。
  - 例：Webアクセスログからユニーク訪問者数を計算する
- **集合演算**：AIやデータ分析の分野で、訓練データとテストデータの集合を扱う際に、差集合を取るなどして重複を防ぐ
  - レコメンドシステムでも、「既にユーザーが購入済みのアイテム集合」と「全アイテム集合」から差集合を取って、未購入アイテムだけを推薦候補にするなど

### **4.4 セットのメリット・デメリット**
- **メリット**
  - 要素の重複を自動的に排除できる
  - 集合演算が簡単に行え、数理的な処理（データ分析など）に強い
  - 要素の存在チェック（`x in set`）がリストに比べて高速（ハッシュテーブルを利用）
- **デメリット**
  - 順序がないため、要素の並び順が必要な場合に向かない
  - リストのようにインデックスでアクセスできない
  - ミュータブル（セット自体は可変）なので、辞書のキーには使えない

---

## **5. 辞書（dict）**

### **5.1 辞書の基本（キーと値）**
辞書（dict）は、キー（key）と値（value）のペアを管理する**連想配列**のようなデータ構造です。キーにはイミュータブル（変更不可）なオブジェクト（文字列、数値、タプルなど）が使えます。値は制限なく、リストや辞書など何でも入ります。

```python
person = {
    "name": "Alice",
    "age": 25,
    "city": "Tokyo"
}
```
- この場合、`"name"`、`"age"`、`"city"` がキー、それぞれに対応する値が `"Alice"`, `25`, `"Tokyo"` となります。

### **5.2 辞書の操作（追加・削除・更新）**
- **要素の追加・更新**
  ```python
  person["job"] = "Engineer"   # キー"job"を追加
  person["age"] = 26           # "age"の値を更新
  ```
- **要素の取得**
  ```python
  print(person["name"])        # Alice
  print(person.get("name"))    # Alice
  print(person.get("unknown", "not found"))  # キーが無い場合、"not found"
  ```
- **要素の削除**
  ```python
  del person["city"]  # "city"キーを削除
  print(person)

  job_val = person.pop("job")
  print(job_val)  # Engineer (popした値)
  ```
- **キーの存在チェック**
  ```python
  if "name" in person:
      print("キー 'name' が存在します")
  ```

### **5.3 代表的なメソッド（keys, values, items, get, pop等）**
| メソッド             | 概要                                                                                  |
|----------------------|---------------------------------------------------------------------------------------|
| `dict.keys()`        | キーの一覧を返す（`dict_keys`オブジェクト）                                          |
| `dict.values()`      | 値の一覧を返す（`dict_values`オブジェクト）                                          |
| `dict.items()`       | (キー, 値) のタプル一覧を返す（`dict_items`オブジェクト）                             |
| `dict.get(key, def)`| キー`key`が存在すれば対応する値を、無ければ`def`を返す                                  |
| `dict.pop(key)`      | キー`key`を削除し、その値を返す（存在しないとKeyError）                                |
| `dict.update(...)`   | 他の辞書やイテラブル(キー:値ペア)で要素を一括更新                                      |

```python
for k, v in person.items():
    print(k, ":", v)
```
このようにして、辞書のすべてのキーと値をイテレートできます。

### **5.4 企業利用例（DBレコード管理、JSONとの親和性）**
- **データベースからの取得結果**: PythonのORM（SQLAlchemyなど）でレコードを取得すると、辞書に近い形でアクセスできる場合がある。データを辞書に格納しておけば、キーで分かりやすく管理・操作できる。
- **JSONとのシームレスな連携**: Web APIではJSON形式でデータを受け渡しすることが多い。Pythonの辞書はJSONと非常に相性が良く、`json`モジュールを使うことで簡単に相互変換が可能。
  ```python
  import json

  data_dict = {"user": "Bob", "score": 85}
  json_str = json.dumps(data_dict)  # 辞書 -> JSON文字列
  print(json_str)  # {"user": "Bob", "score": 85}

  restored_dict = json.loads(json_str)  # JSON文字列 -> 辞書
  print(restored_dict)  # {"user": "Bob", "score": 85}
  ```

- **機械学習・推薦システム**: ユーザーごとの特徴量をキーを"user_id"にして管理したり、アイテム情報を辞書に保持しておくなど、キーと値で明確にデータを紐づけできると扱いやすい。

### **5.5 辞書のメリット・デメリット**
- **メリット**
  - キーによるアクセスが非常に高速（ハッシュテーブルを用いる）
  - JSONと親和性が高く、Webやモバイルとのやり取りに便利
  - Python 3.11系では、辞書の順序保持とわかりやすいエラーメッセージにより、デバッグがさらにしやすくなっている
- **デメリット**
  - キーが変更不可能でなければならない（リストやセットなどは使えない）
  - キーの重複が許されないため、意図せず上書きしてしまうバグに注意
  - 大量データを辞書に詰め込みすぎるとメモリ消費が大きくなる

---

## **6. Python 3.11系でのコレクション周りの注意点・改良点**

### **6.1 エラーメッセージの改良**
Python 3.11系では、コレクション操作のエラーメッセージが一部改善されています。たとえば、辞書で存在しないキーを参照しようとした場合や、タプルで要素を変更しようとした場合に、より詳しい情報や提案が表示されることがあります。

### **6.2 内部実装のパフォーマンス**
- **辞書とセット**はいずれもハッシュテーブルを内部で用いているため、Pythonのバージョンが上がるにつれて微妙な最適化が施されています。
- 頻繁にキーを追加・削除する場合や、大量の要素を保持する場合には、バージョンごとのパフォーマンス差も意識すると良いでしょう。
- また、辞書に順序が保持されるようになった（Python 3.7以降）ことで、特にログ出力やデバッグ時にキーの並び順が固定され、トラブルシュートがしやすくなっています。

---

## **7. コレクション混在の応用例**

### **7.1 リストの中に辞書を入れる**
実務では、「複数のレコードをリストでまとめ、それぞれのレコードを辞書で表現する」というケースがよくあります。

```python
users = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob",   "age": 30},
    {"id": 3, "name": "Charlie", "age": 22}
]

# 全ユーザーの名前を取得
for user in users:
    print(user["name"])
```

### **7.2 辞書の値としてリストやタプルを保持する**
辞書の値は何でも入るので、リストやタプル、別の辞書すら格納できます。

```python
product = {
    "product_id": 1001,
    "name": "Laptop",
    "price": 999.99,
    "tags": ["electronics", "computer", "laptop"],
    "dimensions": (30, 20, 2)  # 幅, 奥行, 高さ
}
```
- 企業のECシステムで、商品情報をこのように辞書に格納し、タグはリストで管理、サイズ情報はタプルにする、など現場ではよくあります。

### **7.3 現場でよくあるデータ構造パターン**
- **リスト×辞書**: 大量の同じ構造のレコードを保持する。データベースのテーブルをイメージ
- **辞書×リスト**: 1つの辞書の値として複数の候補をリストで保持する（例：ユーザーが所有するアイテム一覧）
- **辞書×辞書**: ネストが深くなりすぎると可読性が低下するが、階層的な設定ファイルなどでは有効
- **タプル×辞書**: 座標タプルをキーにして地図情報や行動履歴を格納する、といった応用

---

## **8. 具体的なコード例（大量）**

### **8.1 リスト例**
```python
# リスト操作の一連の流れを通して練習

numbers = [10, 5, 7, 3, 8]
print("初期リスト:", numbers)

# 要素の追加
numbers.append(12)
print("末尾に12を追加:", numbers)

# 要素の挿入
numbers.insert(1, 20)
print("インデックス1に20を挿入:", numbers)

# 要素の削除
numbers.remove(7)
print("値7を削除:", numbers)

# pop()
val = numbers.pop()  # 末尾の要素を取り出し
print("pop()した値:", val)
print("リスト状態:", numbers)

# ソート
numbers.sort()
print("ソート後:", numbers)

# リバース
numbers.reverse()
print("リバース後:", numbers)

# スライス
slice_part = numbers[1:3]
print("スライス [1:3]:", slice_part)

# in 演算子
if 20 in numbers:
    print("20が含まれています")
else:
    print("20は含まれていません")
```

### **8.2 タプル例**
```python
# タプルの利用例

colors = ("red", "green", "blue")
print("タプル colors:", colors)

# インデックス参照
print("colors[0]:", colors[0])

# スライス
print("colors[1:]:", colors[1:])

# 不変性の確認
try:
    colors[1] = "yellow"  # 変更不可なのでエラー
except TypeError as e:
    print("エラー発生:", e)

# 座標タプルの例
coordinate = (100, 200)
print("coordinate:", coordinate)

# 関数からタプルを返す例
def calc_sum_and_product(a, b):
    return (a + b, a * b)

s, p = calc_sum_and_product(3, 5)
print("sum:", s, "product:", p)
```

### **8.3 セット例**
```python
# セット操作の例

s1 = {1, 2, 3, 2, 1}
print("s1:", s1)  # 重複は自動的に除外される {1, 2, 3}

s2 = {3, 4, 5}
print("s2:", s2)

# 要素追加
s1.add(4)
s1.add(2)  # 既にある値を追加→変化なし
print("s1に4と2をadd:", s1)

# 集合演算
union_set = s1.union(s2)
intersection_set = s1 & s2
difference_set = s1 - s2
print("union:", union_set)
print("intersection:", intersection_set)
print("difference:", difference_set)

# removeとdiscardの違い
s1.remove(1)  # 存在しない値をremoveするとKeyError
s1.discard(10)  # 存在しない値でもエラーにならない
print("s1 after remove(1) and discard(10):", s1)
```

### **8.4 辞書例**
```python
# 辞書操作の例

person = {
    "name": "Alice",
    "age": 25,
    "city": "Tokyo"
}
print("初期辞書:", person)

# 追加・更新
person["job"] = "Engineer"
person["age"] = 26
print("追加・更新後:", person)

# get()
name_val = person.get("name", "unknown")
print("person.get('name'):", name_val)

# pop()
job_val = person.pop("job", "no job")
print("pop('job'):", job_val)
print("pop後:", person)

# キー一覧
print("keys:", person.keys())
# 値一覧
print("values:", person.values())
# (キー, 値)一覧
print("items:", person.items())

# キーの存在チェック
if "city" in person:
    print("キー city が存在します")
else:
    print("キー city は存在しません")

# forループで辞書をイテレート
for k, v in person.items():
    print(k, ":", v)
```

### **8.5 応用例：コレクションを使った簡単なアプリケーション**
```python
# ユーザーから複数のデータを受け取り、分析結果を表示する例
# リスト/辞書/セットなどを混在させて利用

def main():
    users = []
    while True:
        name = input("ユーザー名を入力(終了は 'q'): ")
        if name.lower() == 'q':
            break
        age_str = input("年齢を入力: ")
        age = int(age_str)
        # ユーザー情報を辞書としてまとめ、リストに追加
        user_info = {"name": name, "age": age}
        users.append(user_info)

    # ユーザー名のセットを作成して重複をチェック
    name_set = {u["name"] for u in users}
    if len(users) != len(name_set):
        print("重複したユーザー名があります")
    else:
        print("全ユーザー名は一意です")

    # 年齢の平均を求める
    if users:
        avg_age = sum(u["age"] for u in users) / len(users)
        print(f"ユーザー数: {len(users)}, 平均年齢: {avg_age:.2f}")
    else:
        print("ユーザーが1人も登録されていません")

if __name__ == "__main__":
    main()
```
このように、**リスト**を使って複数ユーザーを管理し、**辞書**でユーザー情報をまと め、**セット**を使って重複確認をする、といった形でコレクションを組み合わせると、実務でもよくある機能を短いコードで実装できます。

---

## **9. 演習問題**

ここでは、コレクションを扱う一連の演習問題を提示します。自分で解いてみた後、解答例を参照してください。

### **9.1 問題文**

#### **問題1: リスト操作の総合演習**
1. リスト `numbers` を `[10, 20, 30, 40, 50]` で初期化する。
2. このリストの末尾に `60` を追加し、結果を表示する。
3. インデックス2番目（0-basedで3番目の要素）に `25` を挿入し、結果を表示する。
4. 値 `40` を `remove()` で削除し、結果を表示する。
5. `pop()` で末尾要素を取り出して表示し、リストの状態を確認する。
6. リストを降順（大きい値が先頭）にソートして表示する。
7. すべての要素を合計して表示する（組み込み関数 `sum()` を使用しても良い）。

#### **問題2: タプルでイミュータブルを体験する**
1. タプル `colors` を `("red", "green", "blue")` で宣言する。
2. `colors[1]` を `"yellow"` に変更してみて、エラーが発生することを確認する。
3. タプルのスライスを使って、`("green", "blue")` を取り出し表示する。
4. 関数からタプル `(最小値, 最大値)` を返す仕組みを作り、任意のリストに対して呼び出して結果を表示する。

#### **問題3: セットでユニークな要素を扱う**
1. セット `s1` を `{1, 2, 3, 2, 1}` で初期化し、出力結果を確認する（重複が削除される）。
2. `s2 = {2, 3, 4}` を作成し、`s1` と `s2` の和集合・積集合・差集合をそれぞれ表示する。
3. `s1.add(5)` で要素を追加し、結果を表示する。
4. `s1.remove(2)` と `s1.discard(10)` を行い、結果を表示する。

#### **問題4: 辞書を活用した顧客管理**
1. 辞書 `customer` を作り、以下のキーと値を設定する。
   - `"id"`: 101
   - `"name"`: `"Alice"`
   - `"purchase"`: `[("itemA", 2), ("itemB", 1)]`  ※購入商品と購入数をタプルのリストで管理
2. `customer["age"] = 30` を追加し、辞書の内容を表示する。
3. `purchase` リストを1件追加する。たとえば `("itemC", 4)`。
4. `for` ループで `purchase` の中身を取り出し、商品名と数を表示する。
5. キー `"purchase_total"` に購入数の合計を計算して追加し、最終的な辞書の内容を表示する。

#### **問題5: 応用（コレクション混在）**
- 以下のような構造を持つ辞書 `shop_data` を作成する。
  - "shop_id": 1001
  - "name": "Sample Store"
  - "categories": ["Electronics", "Books", "Toys"]   (リスト)
  - "location": (35.6895, 139.6917)   (タプル, 東京駅あたりの緯度経度を仮に使用)
  - "inventory": セットで `{101, 102, 103}` (商品IDの在庫リストとし、重複なし)
- この辞書を用いて、以下の処理を行うプログラムを作る。
  1. `categories` に `"Clothing"` を追加して表示する。
  2. `inventory` に `104` を `add` し、`102` を `remove` して表示する。
  3. `location` のタプルをそのまま表示し、`location[0]`（緯度）のみ取り出して表示する。
  4. 辞書のすべてのキーと値を `for k, v in shop_data.items(): ...` の形で表示する。

---

### **9.2 解答例**

#### **問題1の解答例**
```python
def solve_problem1():
    # 1
    numbers = [10, 20, 30, 40, 50]
    print("初期リスト:", numbers)

    # 2
    numbers.append(60)
    print("末尾に60を追加:", numbers)

    # 3
    numbers.insert(2, 25)
    print("インデックス2に25を挿入:", numbers)

    # 4
    numbers.remove(40)
    print("値40をremove:", numbers)

    # 5
    popped_val = numbers.pop()
    print("pop()で取り出した値:", popped_val)
    print("pop後のリスト:", numbers)

    # 6
    numbers.sort(reverse=True)  # 降順ソート
    print("降順ソート:", numbers)

    # 7
    total = sum(numbers)
    print("リストの合計値:", total)


if __name__ == "__main__":
    solve_problem1()
```

#### **問題2の解答例**
```python
def solve_problem2():
    # 1
    colors = ("red", "green", "blue")
    print("タプル colors:", colors)

    # 2
    try:
        colors[1] = "yellow"  # エラー
    except TypeError as e:
        print("タプルは変更不可:", e)

    # 3
    print("スライス [1:]:", colors[1:])  # ("green", "blue")

    # 4
    def get_min_max(data):
        return (min(data), max(data))

    sample_list = [5, 2, 9, 1, 7]
    min_val, max_val = get_min_max(sample_list)
    print(f"リスト {sample_list} の最小値={min_val}, 最大値={max_val}")

if __name__ == "__main__":
    solve_problem2()
```

#### **問題3の解答例**
```python
def solve_problem3():
    # 1
    s1 = {1, 2, 3, 2, 1}
    print("s1:", s1)

    # 2
    s2 = {2, 3, 4}
    print("s2:", s2)
    print("和集合 (s1 | s2):", s1 | s2)
    print("積集合 (s1 & s2):", s1 & s2)
    print("差集合 (s1 - s2):", s1 - s2)

    # 3
    s1.add(5)
    print("s1に5を追加:", s1)

    # 4
    s1.remove(2)
    s1.discard(10)
    print("s1.remove(2) と s1.discard(10) 後:", s1)

if __name__ == "__main__":
    solve_problem3()
```

#### **問題4の解答例**
```python
def solve_problem4():
    # 1
    customer = {
        "id": 101,
        "name": "Alice",
        "purchase": [("itemA", 2), ("itemB", 1)]
    }
    print("初期辞書:", customer)

    # 2
    customer["age"] = 30
    print("年齢を追加:", customer)

    # 3
    customer["purchase"].append(("itemC", 4))
    print("purchaseにitemCを追加:", customer["purchase"])

    # 4
    print("購入商品リスト:")
    for item, qty in customer["purchase"]:
        print(f"  {item} x {qty}")

    # 5
    total_qty = sum(qty for (_, qty) in customer["purchase"])
    customer["purchase_total"] = total_qty
    print("購入数の合計をpurchase_totalに追加:", customer)

if __name__ == "__main__":
    solve_problem4()
```

#### **問題5の解答例**
```python
def solve_problem5():
    # データ構造作成
    shop_data = {
        "shop_id": 1001,
        "name": "Sample Store",
        "categories": ["Electronics", "Books", "Toys"],
        "location": (35.6895, 139.6917),
        "inventory": {101, 102, 103}
    }

    # 1. categoriesに"Clothing"を追加
    shop_data["categories"].append("Clothing")
    print("categoriesを追加:", shop_data["categories"])

    # 2. inventory に 104 を add, 102 を remove
    shop_data["inventory"].add(104)
    shop_data["inventory"].remove(102)
    print("inventory 更新後:", shop_data["inventory"])

    # 3. location のタプルを表示し、緯度のみ取得
    print("location:", shop_data["location"])
    print("location[0] (緯度):", shop_data["location"][0])

    # 4. 辞書のキーと値を全て表示
    for k, v in shop_data.items():
        print(k, ":", v)

if __name__ == "__main__":
    solve_problem5()
```

---

## **10. まとめ**

### **10.1 コレクションを使いこなすポイント**
- **リスト**：可変長、順序あり、追加・削除・ソートがしやすい。大量データのバッチ処理や解析によく使われる。
- **タプル**：不変、座標や一時的な複数返却など、安全性を重視したい場面で活用。
- **セット**：重複なし、順序なし。集合演算やユニーク要素の抽出に有効。
- **辞書**：キーと値のマッピング。データベースレコードやJSONとの相性が良い。キー検索が高速で、大規模データでも使いやすい。

現場での開発でも、これらを状況に応じて組み合わせることで、効率の良いデータ管理・処理が可能になります。

### **10.2 今後の学習・応用について**
- 次のステップでは、これらコレクションを**関数やクラス**で使いこなし、より大規模なコードへ展開していきます。
- **ファイル操作**や**データベース連携**、そして**機械学習**や**推薦システム**などでも基本的にはコレクションの考え方が必須となります。
- Python 3.11系での微細な改善点（エラーメッセージやパフォーマンス最適化）を活かしながら、実際にコーディング量をこなして理解を深めましょう。

---

## **11. 参考文献・参考URL**

- **公式ドキュメント**: [Python 3.11 Documentation](https://docs.python.org/3.11/)
- **PEP 585**（List, tuple, dict, setに対するジェネリック型ヒント導入）
- **PEP 468**（kwargsの順序保持、3.6〜3.7にかけての辞書順序の標準化）
- **Pythonチュートリアル**: コレクションリファレンス
- **Python実践入門**（書籍）: コレクションとファイル操作の章

企業での利用例は、各業界のPython導入事例（Web検索やカンファレンスレポートなど）を参照するとより具体的なものを得られます。

---

# **（付録）文字数カウントについて**
本資料は、講義内容を詳述し、コード例や解説を多数盛り込むことで、**30,000文字以上**を目指して記述しています。ChatGPTの出力がその条件を満たしているか、エディタや文字数カウンター等でご確認ください。

以上で「第5回：コレクション（リスト、タプル、セット、辞書）の使い方」の講義資料となります。時間配分としては、1時間から1時間半の枠内で、要所要所のコード実演や演習を行うと効果的です。演習時間は参加者のレベルや質問量に応じて調整してください。コレクションの理解はPythonプログラミングの大きな基盤になるため、ぜひ丁寧に練習を重ねてください。